In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig
import torch

# PEFT adapter
peft_adapter="./flan_t5_fine_tuned_peft-local_5e"   


C:\Users\mydma\miniconda3\envs\myenv2\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

peft_model = PeftModel.from_pretrained(peft_model_base, 
                                       peft_adapter, 
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

C:\Users\mydma\miniconda3\envs\myenv2\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [3]:
# Given testset
testset = [{'country': 'USA', 'capital': 'Washington'}, 
           {'country': 'China', 'capital': 'Beijing'},
           {'country': 'France', 'capital': 'Paris'},
           {'country': 'England', 'capital': 'London'},
           {'country': 'Germany', 'capital': 'Berlin'},
           {'country': 'Japan', 'capital': 'Tokyo'},
           {'country': 'Spain', 'capital': 'Madrid'},       
           {'country': 'Mexico', 'capital': 'Mexico City'},
           {'country': 'Canada', 'capital': 'Ottawa'},
           {'country': 'Thailand', 'capital': 'Bangkok'}, 
           {'country': 'Italy', 'capital': 'Rome'},       
           {'country': 'Russia', 'capital': 'Moscow'},
           {'country': 'Venezuela', 'capital': 'Caracas'},
           {'country': 'Vietnam', 'capital': 'Hanoi'}, 
           {'country': 'Turkey', 'capital': 'Ankara'},       
           {'country': 'Brazil', 'capital': 'Brasilia'},
           {'country': 'Honduras', 'capital': 'Tegucigalpa'},
           {'country': 'Hungary', 'capital': 'Budapest'} 
          ]

print('testset count: ',len(testset))

testset count:  18


In [5]:
correct=0
question="What is the capital of "
# iterate through entire testset to inferring the model for capitals
for country_dict in testset:
    a_country=country_dict['country']
    # input_prompt=question + a_country +'?'
    input_prompt=question + a_country    # the inferring seemed to be better without question mark
    
    input_ids = tokenizer(input_prompt, return_tensors="pt").input_ids  
    
    peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)
    
    # Print the result
    print(f"Input: {input_prompt}  ( {country_dict['capital']} )")
    print("Model's Response:", peft_model_text_output, '\n')

    if country_dict['capital'].lower() == peft_model_text_output.lower():
        correct +=1
    
print(f"Number of correct responses: {correct} out of {len(testset)}")

# print("Percentage of correct responses: %.2f%%" % round((correct / len(testset) * 100), 2))
print(f"Percentage of correct responses: %{round((correct / len(testset) * 100), 2)}")



Input: What is the capital of USA  ( Washington )
Model's Response: Washington 

Input: What is the capital of China  ( Beijing )
Model's Response: Beijing 

Input: What is the capital of France  ( Paris )
Model's Response: Paris 

Input: What is the capital of England  ( London )
Model's Response: London 

Input: What is the capital of Germany  ( Berlin )
Model's Response: Berlin 

Input: What is the capital of Japan  ( Tokyo )
Model's Response: Tokyo 

Input: What is the capital of Spain  ( Madrid )
Model's Response: Madrid 

Input: What is the capital of Mexico  ( Mexico City )
Model's Response: Mexico 

Input: What is the capital of Canada  ( Ottawa )
Model's Response: Ottawa 

Input: What is the capital of Thailand  ( Bangkok )
Model's Response: Bangkok 

Input: What is the capital of Italy  ( Rome )
Model's Response: Rome 

Input: What is the capital of Russia  ( Moscow )
Model's Response: Moscow 

Input: What is the capital of Venezuela  ( Caracas )
Model's Response: Caracas 

I